In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import os

In [2]:
networks_path="/Users/orsi/Dropbox/moby-data_not_shared/networks/"
path_out='/Users/orsi/Dropbox/moby-data_not_shared/SMALL_data/ASSORTATIVITY/ASS_FEMALE/'
p_out2="/Users/orsi/Dropbox/moby-data_not_shared/SMALL_data/NEW_SEGR_NOV/"

In [6]:
p_out='/Users/orsi/Dropbox/moby-data_not_shared/SMALL_data/ASSORTATIVITY/'
moby=pd.read_csv('/Users/orsi/Dropbox/moby-data_not_shared/new_moby_from09092019.csv').set_index('game_id')
old_inc=pd.read_csv(p_out2+"NEW_segregation_pessimistic8009.csv").set_index('gamie_id')
new_inc=pd.read_csv(p_out+"ASSORTATIVITY8009.csv").set_index('game_id')
MOBY=moby.join(new_inc, lsuffix='_new').join(old_inc, lsuffix='_old')

In [7]:
MOBY=MOBY[MOBY["num_edges"]<=2000]
MOBY=MOBY[MOBY["num_edges"]>0]

In [8]:
moby_filter = MOBY[(MOBY["prop_unknown"]<0.5) 
                   & (MOBY["female"]>=1) 
                   & (MOBY["Net_size"]>=3)]

In [9]:
len(moby_filter)

4011

In [10]:
def calc_net(G,G_df, game_id):
    res['assortativity']=assortativity
    res['game_id']=game_id
    return res

In [11]:
def join_gender_data(data, genders):
    data1=data.merge(genders[['developer_id', 'gender']], left_on='dev1', right_on='developer_id')
    data2=data1.merge(genders[['developer_id', 'gender']], left_on='dev2', right_on='developer_id')
    data3=data2.rename(columns={'gender_x':'dev1_gender',
                        'gender_y':'dev2_gender'})
    data4=data3[['dev1', 'dev2', 'weight','dev1_gender', 'dev2_gender' ]]
    return data4


def create_gender_con(row):
    if row['dev1_gender']=='male' and row['dev2_gender']=='male':
        return 'mm'
    elif row['dev1_gender']=='female' and row['dev2_gender']=='female':
        return 'ff'
    if row['dev1_gender']=='female' and row['dev2_gender']=='male' or row['dev1_gender']=='male' and row['dev2_gender']=='female':
        return 'mixed'
    
def prepare_data(g):
    df=nx.to_pandas_edgelist(g, source='dev1', target='dev2')
    node_genders=pd.DataFrame.from_dict(dict(g.nodes(data=True))).T
    node_genders['developer_id']=node_genders.index
    df2=join_gender_data(df, node_genders)
    df2['gender_con']=df2.apply(create_gender_con,1)
    return df2

def get_nodes(df):
    return list(set(df["dev1"]).union(set(df["dev2"])))


def calc_cross_calls(df):
    if 'mixed' in df['gender_con'].value_counts().index:
        return len(df[df['gender_con']=='mixed'])
    else:
        return 0


In [12]:
def get_unique_users(data, g_dict):
    u=pd.DataFrame(get_nodes(data)).set_index(0).join(pd.DataFrame(g_dict.items()).set_index(0), how='inner')
    return u

In [13]:
def coreness_gender(g2, g_dict_change):
    try:
        core_nodes=nx.distance_measures.center(g2)
        periphery_nodes=nx.distance_measures.periphery(g2)
        core_genders=[]
        per_genders=[]
        for cn in core_nodes:
            core_genders.append(g_dict_change[cn])
            res=dict(Counter(core_genders))
        for pn in periphery_nodes:
            per_genders.append(g_dict_change[pn])
            res2=dict(Counter(per_genders))
        return (res.get('male'),res.get('female'),res2.get('male'),res2.get('female'), len(core_nodes), len(periphery_nodes))
    except:
        Gcc = sorted(nx.connected_components(g2), key=len, reverse=True)
        giant = g2.subgraph(Gcc[0])
        core_nodes=nx.distance_measures.center(giant)
        periphery_nodes=nx.distance_measures.periphery(giant)
        core_genders=[]
        per_genders=[]
        for cn in core_nodes:
            core_genders.append(g_dict_change[cn])
            res=dict(Counter(core_genders))
        for pn in periphery_nodes:
            per_genders.append(g_dict_change[pn])
            res2=dict(Counter(per_genders))
        return (res.get('male'),res.get('female'),res2.get('male'),res2.get('female'), len(core_nodes), len(periphery_nodes))
        #except:
        #    return (np.nan,np.nan,np.nan,np.nan, np.nan, np.nan)

In [14]:
genders_original=pd.read_csv('/Users/orsi/Dropbox/moby-data_not_shared/user_credits_game_id_year_gender_score.csv')

In [18]:
nets=os.listdir(networks_path)
game_ids=list(moby_filter.index)

In [20]:
g_dict=genders_original.set_index('developer_id')['gender'].to_dict()

In [21]:
len(game_ids)

4011

In [69]:
def change_gender_ratio(node_genders, f):
    interim=node_genders[node_genders['gender']=='unknown'].sample(frac=f).replace('unknwon','female')
    int2=pd.DataFrame()
    for index, row in node_genders.iterrows():
        if index not in interim.index:
            int2=int2.append(row) 
    int2=int2.replace('unknown', 'male')
    return int2

In [85]:
node_genders_new['gender'].value_counts()['male']

4

In [86]:
def calc_all_network_metric(df, node_genders_new):
    df2=join_gender_data(df, node_genders_new)
    df2['gender_con']=df2.apply(create_gender_con,1)
    cross_edge_ratio=calc_cross_calls(df2)
    ASS=nx.attribute_assortativity_coefficient(G, "gender")
    genders_net=get_unique_users(data2, g_dict)[1].values
    all_genders_game=list(get_unique_users(data, g_dict)[1])
    c={}
    #EDGES
    try:
        c['n_males']=node_genders_new['gender'].value_counts()['male']
    except:
        c['n_males']=0
    try:
        c['n_females']=node_genders_new['gender'].value_counts()['female']
    except:
        c['n_females']=0
    c['assortativity']=ASS
    c['cross_edge_ratio']=cross_edge_ratio
    c["male_male"]=len(df2[df2['gender_con']=='mm'])
    c["female_female"]=len(df2[df2['gender_con']=='ff'])
    c["mixed"]=len(df2[df2['gender_con']=='mixed'])
    c["male_male_w"]=len(df2[df2['gender_con']=='mm'])
    c["female_female_w"]=len(df2[df2['gender_con']=='ff'])
    c["mixed_w"]=df2[df2['gender_con']=='mixed']['weight'].sum()
    c["num_edges"]=len(df2)
    c["num_Weighted_edges"]=df2['weight'].sum()
    c["Avg_Weight"]=df2['weight'].mean()
    male_coreN,female_coreN,male_peripheryN,female_peripheryN, num_core,num_periphery =coreness_gender(G, g_dict)
    c["male_coreN"]=male_coreN
    c["female_coreN"]=female_coreN
    c["male_peripheryN"]=male_peripheryN
    c["female_peripheryN"]=female_peripheryN
    c["num_core"]=num_core
    c["num_periphery"]=num_periphery
    c["Net_size"]=len(genders_net)
    c["Team_size"]=len(all_genders_game)
    c['gamie_id']=game_id
    return pd.DataFrame.from_dict(c, 'index').T

In [102]:
res=pd.DataFrame()
j=0
for i in nets:
    j+=1
    game_id=int(i.split(".")[0])
    if game_id in game_ids:
        print(i)
        if j%500==0:
            print(j)
            res.to_csv(path_out+"ASSORTATIVITY_FEMALE"+str(j)+'.csv')
        data=pd.read_csv(networks_path+i, sep=",")
        if len(data)>0:
            data1=data.loc[~(data["dev1"] == data["dev2"])]
            data2=data1[data1['weight'] > 0]
            if len(data2)>3:
                G=nx.from_pandas_edgelist(data2, source='dev1', target='dev2', edge_attr='weight')
                nx.set_node_attributes(G, g_dict, 'gender')
                #df=prepare_data(G)
                df=nx.to_pandas_edgelist(G, source='dev1', target='dev2')
                node_genders=pd.DataFrame.from_dict(dict(G.nodes(data=True))).T
                node_genders['developer_id']=node_genders.index
                n_unknown=len(node_genders[node_genders['gender']=='unknown'])
                for f in [0.25, 0.5, 0.75]:
                    for i in range(0,10):
                        print(i)
                        node_genders_new=change_gender_ratio(node_genders, f) # f is the fraction of unknowns who is changed to female, rest to male
                        r=calc_all_network_metric(df2, node_genders_new)
                        r["i"]=i
                        r['f']=f
                        res=res.append(r)
res.to_csv(path_out+"ASSORTATIVITY_FEMALE"+str(j)+'.csv')

15145.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
1859.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
43767.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10766.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
14594.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
13084.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
13882.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
36019.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
11808.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
31224.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
23444.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
327.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
5580.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
21777.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7


/Users/orsi/anaconda3/lib/python3.7/site-packages/networkx/algorithms/assortativity/correlation.py:263: RuntimeWarning: invalid value encountered in double_scalars
  r = (t - s) / (1 - s)


1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
16622.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
31118.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
7958.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
41806.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
4228.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
3780.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
25295.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
4641.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
35352.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
27577.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
6572.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
3314.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
5641.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
23785.0.csv
0

/Users/orsi/anaconda3/lib/python3.7/site-packages/networkx/algorithms/assortativity/correlation.py:263: RuntimeWarning: invalid value encountered in double_scalars
  r = (t - s) / (1 - s)


1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
23153.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
23910.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
2156.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
1265.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
28620.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
43270.0.csv
0


...

1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
17000.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
7739.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
639.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
8101.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
29947.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
34119.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
13666.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
5898.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
11200.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
29651.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
9654.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
4437.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
3127.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
9179.0.csv
0
1

/Users/orsi/anaconda3/lib/python3.7/site-packages/networkx/algorithms/assortativity/correlation.py:263: RuntimeWarning: invalid value encountered in double_scalars
  r = (t - s) / (1 - s)


1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
11509.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
5413.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
665.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
8118.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
620.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
26725.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
21235.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
42260.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
8635.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
13911.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
16734.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
15895.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
15407.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
38363.0.csv
0

/Users/orsi/anaconda3/lib/python3.7/site-packages/networkx/algorithms/assortativity/correlation.py:263: RuntimeWarning: invalid value encountered in double_scalars
  r = (t - s) / (1 - s)


1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
4455.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
4084.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
30765.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
3943.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
4410.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
2145.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
14329.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
24415.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
5410.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
8572.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
15740.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
37659.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
30798.0.csv
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
15705.0.csv
0

In [104]:
res.to_csv(path_out+"ASSORTATIVITY_FEMALE"+str(j)+'.csv')

In [43]:
female_p_in='/Users/orsi/Dropbox/moby-data_not_shared/SMALL_data/ASSORTATIVITY/ASS_FEMALE/'
male_p_in='/Users/orsi/Dropbox/moby-data_not_shared/SMALL_data/ASSORTATIVITY/ASS_MALE/'

In [91]:
len(res)

131542

In [97]:
res.groupby('f')["i"].value_counts()

f     i  
0.25  0      435
      1      435
      2      435
      3      435
      4      435
            ... 
0.75  96     434
      97     434
      98     434
      99     434
      100    434
Name: i, Length: 303, dtype: int64

In [98]:
len(res['gamie_id'].value_counts())

435

In [100]:
res[res['gamie_id']==6449].groupby('f')['assortativity'].mean()

f
0.25   -0.073171
0.50   -0.073171
0.75   -0.073171
Name: assortativity, dtype: float64

In [101]:
res.to_csv(path_out+"ASSORTATIVITY_FEMALE"+str(j)+'.csv')

In [5]:
50074*0.005

250.37